In [1]:
import numpy as np
import xarray as xr
import glob
# To convert coordinates from lat, lon to polar stereo x,y 
from pyproj import Transformer
# KD Tree for nearest neighbours
from scipy.spatial import cKDTree


In [2]:
import numpy as np
import xarray as xr    
#import cartopy.crs as ccrs
#import matplotlib.ticker as mticker
#import matplotlib.pyplot as plt
#from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
# To interpolate NEMO data from Nico's mask files to the NEMO simulation grid 
from scipy.interpolate import griddata
import itertools
# To convert coordinates from lat, lon to polar stereo x,y 
from pyproj import Transformer
# KD Tree for nearest neighbours
from scipy.spatial import cKDTree
# To play a sound when at the end of the code you want to run 
import os
##os.system("printf '\a'")
import time
import glob

In [122]:
nemo_run = 'isfru94'
#nemo_run = 'OPM016'
# Options are 'OPM016', 'OPM018', 'OPM021', 'ctrl94', 'isf94', 'isfru94'

In [123]:
# Set all the filepaths that will be required 
filepath_base = '/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/'
# Basin numbers 
filepath_basin_nos = filepath_base + 'MASKS_PARAM/basin_raster_extrapolate_eANT025.nc'
# CFG for NEMO 1 degree
filepath_cfg_reduced_cav = filepath_base + 'MASKS_PARAM/domain_cfg_eANT025.L121_REDUCED_CAV.nc'


In [124]:
# Set all the filepaths that will be required 
if nemo_run == 'OPM016':
    filepath_nemo = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/NEMO_eORCA025.L121-OPM016/'
    file_extension = '*gridT*.nc'
    filepath_mesh_mask = filepath_nemo + 'eORCA025.L121-' + nemo_run + '_mesh_mask.nc'
elif nemo_run == 'OPM018':
    filepath_nemo = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/NEMO_eORCA025.L121-OPM018/'
    file_extension = '*gridT*.nc'
    filepath_mesh_mask = filepath_nemo + 'eORCA025.L121-' + nemo_run + '_mesh_mask.nc'
elif nemo_run == 'OPM021':
    filepath_nemo = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/NEMO_eORCA025.L121-OPM021/'
    file_extension = '*gridT*.nc'
    filepath_mesh_mask = filepath_nemo + 'eORCA025.L121-' + nemo_run + '_mesh_mask.nc'
elif nemo_run == 'ctrl94':
    filepath_nemo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/CHRISTOPH_DATA/'
    filepath_extension = '*.nc'
    filepath_mesh_mask = filepath_nemo + 'mesh_mask.nc'
    filepath_cfg = filepath_nemo + 'domain_cfg_eANT025.L121.nc'
elif nemo_run == 'isf94':
    filepath_nemo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/CHRISTOPH_DATA/'
    filepath_extension = '*.nc'
    filepath_mesh_mask = filepath_nemo + 'mesh_mask.nc'
    filepath_cfg = filepath_nemo + 'domain_cfg_eANT025.L121.nc'
elif nemo_run == 'isfru94':
    filepath_nemo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/CHRISTOPH_DATA/'
    filepath_extension = '*.nc'
    filepath_mesh_mask = filepath_nemo + 'mesh_mask.nc'
    filepath_cfg = filepath_nemo + 'domain_cfg_eANT025.L121.nc'
else:
    filepath_pierre = []
    file_extension = []
    print('Help, I do not know this nemo_run')
print(nemo_run, filepath_nemo)

if len(glob.glob(filepath_mesh_mask)) != 1:
    print('WARNING: mesh_mask is not in the specified location')

isfru94 /bettik/burgardc/DATA/SUMMER_PAPER/raw/CHRISTOPH_DATA/


In [125]:
#fp = '/bettik/burgardc/DATA/BASAL_MELT_PARAM/raw/NEMO_eORCA025.L121-OPM016/' + 'eORCA025.L121-' + 'OPM016' + '_mesh_mask.nc'
#fpxr = xr.load_dataset(fp)
#fpmm = xr.load_dataset(filepath_mesh_mask)
#fpcfg = xr.load_dataset(filepath_nemo + 'domain_cfg_eANT025.L121.nc')

In [126]:
#fpmm

In [127]:
#fpxr

In [128]:
#for i in list(fpmm.data_vars):
#    if i not in list(fpcfg.data_vars):
#        print(i)
#print('FLIP')
#for i in list(fpcfg.data_vars):
#    if i not in list(fpmm.data_vars):
#        print(i)

In [129]:
# Load in the basin numbers file 
ds0=xr.open_dataset(filepath_basin_nos)
# Load in the domain geometry files 
dsREF=xr.open_dataset(filepath_mesh_mask)
dsREDCAV=xr.open_dataset(filepath_cfg_reduced_cav)
if 'time_counter' in list(dsREF.dims):
    print('Squeezing the time_counter level in the cfg file')
    dsREF = dsREF.squeeze('time_counter')
if dsREF.dims['y'] == 1207:
    print('Reducing y dimension')
    min_lat = -52.2
    y_trim = np.max(np.argwhere(dsREF.nav_lat[:,0].data <= min_lat))
    # Take this slice 
    dsREF = dsREF.sel(y = slice(0,y_trim+1))
    dsREF = dsREF.sel(x = slice(1,1441))
dsREDCAV = dsREDCAV.squeeze('time_counter')

if nemo_run in ('ctrl94', 'isf94', 'isfru94'):
    dsCFG = xr.open_dataset(filepath_cfg)
    if 'time_counter' in list(dsCFG.dims):
        print('Squeezing the time_counter level in the cfg file')
        dsCFG = dsCFG.squeeze('time_counter')
    print('Loaded a separate CFG file in addition to the mesh mask')

Squeezing the time_counter level in the cfg file
Squeezing the time_counter level in the cfg file
Loaded a separate CFG file in addition to the mesh mask


In [130]:
# Create masks for the ocean, closed cavities, open cavities, grounded ice 
if nemo_run in ('ctrl94', 'isf94', 'isfru94'):
    ocean           = xr.where(((dsREF.mbathy > 1.5) & (dsREF.misf > 0.5) & (dsREF.misf < 1.5)), 1, 0)
    ocean_nan       = xr.where(((dsREF.mbathy > 1.5) & (dsREF.misf > 0.5) & (dsREF.misf < 1.5)), 1, np.nan)
else:
    ocean           = xr.where(((dsREF.misf>0.5)&(dsREF.misf<1.5)),1,0)
    ocean_nan       = xr.where(((dsREF.misf>0.5)&(dsREF.misf<1.5)),1,np.nan)
all_cavities        = xr.where((dsREF.misf>1.5),1,0)
all_cavities_nan    = xr.where((dsREF.misf>1.5),1,np.nan)
closed_cavities     = xr.where(((dsREF.misf>1.5)&(dsREDCAV.top_level<0.5)),1,0)
closed_cavities_nan = xr.where(((dsREF.misf>1.5)&(dsREDCAV.top_level<0.5)),1,np.nan)
open_cavities       = xr.where((dsREDCAV.top_level>1.5),1,0)
open_cavities_nan   = xr.where((dsREDCAV.top_level>1.5),1,np.nan)
open_cav_ocean      = xr.where((ocean > 0.5),1, 0) + xr.where((open_cavities > 0.5), 1, 0)
open_cav_ocean_nan  = xr.where((open_cav_ocean == 1), 1, np.nan)
grounded_ice        = xr.where((ocean+all_cavities) == 0, 1,0)
grounded_ice_nan    = xr.where((ocean+all_cavities) == 0, 1,np.nan)
# Create the field for the basin numbers 
basins_NEMO         = ds0.basin
# Extract other useful paramaters
areas = dsREDCAV.e1t*dsREDCAV.e2t
if nemo_run in ('ctrl94', 'isf94', 'isfru94'):
    bathymetry = dsCFG.bathy_metry
    isf_draft = dsCFG.isf_draft
else:
    bathymetry = dsREF.bathy_metry
    isf_draft = dsREF.isfdraft

In [131]:
# Create files of distances and slopes
# Create a pyproj transform to go from lat lon to polar stereo
transformer = Transformer.from_crs("epsg:4326","epsg:3031")

# Create a list of all the points in closed ice shelves 
CC_lat, CC_lon = dsREF.nav_lat.data[closed_cavities == 1], dsREF.nav_lon.data[closed_cavities == 1]
CC_x, CC_y = transformer.transform(CC_lat, CC_lon)
points_CC = np.array(list(zip(CC_x, CC_y)))
# Create a list of all the grounded ice points
GL_lat, GL_lon = dsREF.nav_lat.data[grounded_ice == 1], dsREF.nav_lon.data[grounded_ice == 1]
GL_x, GL_y = transformer.transform(GL_lat, GL_lon)
points_GL = np.array(list(zip(GL_x, GL_y)))
# Create a list of all the open ocean points 
OO_lat, OO_lon = dsREF.nav_lat.data[ocean == 1], dsREF.nav_lon.data[ocean == 1]
OO_x, OO_y = transformer.transform(OO_lat, OO_lon)
points_OO = np.array(list(zip(OO_x, OO_y)))
# Create a list of all the open ocean cavity points
OC_lat, OC_lon = dsREF.nav_lat.data[open_cav_ocean == 1], dsREF.nav_lon.data[open_cav_ocean == 1]
OC_x, OC_y = transformer.transform(OC_lat, OC_lon)
points_OC = np.array(list(zip(OC_x, OC_y)))

# Create the cKD Tree for the grounding line 
tree = cKDTree(points_GL)
# Query the cKD Tree for the closed cavity points 
dist, index = tree.query(points_CC, k = 1)
distances_GL = np.ones(dsREF.nav_lat.data.shape)*np.nan
distances_GL[closed_cavities == 1] = dist

# Create the cKD Tree for the open ocean
tree = cKDTree(points_OO)
# Query the cKD Tree for the closed cavity points 
dist, index = tree.query(points_CC, k = 1)
distances_OO = np.ones(dsREF.nav_lat.data.shape)*np.nan
distances_OO[closed_cavities == 1] = dist

# Create the cKD Tree for the open ocean and cavities
tree = cKDTree(points_OC)
# Query the cKD Tree for the closed cavity points 
dist, index = tree.query(points_CC, k = 1)
distances_OC = np.ones(dsREF.nav_lat.data.shape)*np.nan
distances_OC[closed_cavities == 1] = dist

In [132]:
#print(len(np.unique(index)))
nearest_lats = np.ones(len(np.unique(index)))
nearest_lons = np.ones(len(np.unique(index)))
for i in range(len(np.unique(index))):
    nearest_lats[i], nearest_lons[i] = dsREF.nav_lat.values[open_cav_ocean == 1][np.unique(index)[i]], \
                                       dsREF.nav_lon.values[open_cav_ocean == 1][np.unique(index)[i]]
    print(i+1, 'out of', len(np.unique(index)), 'Coordinates calculated', end = '\r')
for i in range(len(nearest_lats)):
    if i == 0:
        mask = (dsREF.nav_lat == nearest_lats[i]) & (dsREF.nav_lon == nearest_lons[i])
        print('')
    else:
        mask_this_point = (dsREF.nav_lat == nearest_lats[i]) & (dsREF.nav_lon == nearest_lons[i])
        mask = mask + mask_this_point
    print(i+1, 'out of', len(np.unique(index)), 'Added to mask', end = '\r')
mask_TS_profiles = mask
mask_TS_profiles_nan = xr.where(mask_TS_profiles == 1, 1, np.nan)

2230 out of 2230 Coordinates calculated


In [133]:
# Create a pyproj transform to go from lat lon to polar stereo
transformer = Transformer.from_crs("epsg:4326","epsg:3031")

# In order to calculate the slopes we need the distance between points, 
# and because the points are on the NEMO lat-lon grid we need to convert them to polar stereo the calculate the distances 
# Create a pyproj transformer to transform onto polar stereo
lat, lon = dsREF.nav_lat.data, dsREF.nav_lon.data
grid_x, grid_y = transformer.transform(lat, lon)
# Use the np.roll function to move the data slightly 
shifted_x_p0 = (grid_x - np.roll(grid_x, shift = 1, axis = 0))
shifted_x_m0 = (grid_x - np.roll(grid_x, shift = -1, axis = 0))
shifted_x_p1 = (grid_x - np.roll(grid_x, shift = 1, axis = 1))
shifted_x_m1 = (grid_x - np.roll(grid_x, shift = -1, axis = 1))
shifted_y_p0 = (grid_y - np.roll(grid_y, shift = 1, axis = 0))
shifted_y_m0 = (grid_y - np.roll(grid_y, shift = -1, axis = 0))
shifted_y_p1 = (grid_y - np.roll(grid_y, shift = 1, axis = 1))
shifted_y_m1 = (grid_y - np.roll(grid_y, shift = -1, axis = 1))
# Because you need to calculate the hypotenuse distance you need x and y for both directions of shift 
shifted_p_lon = np.sqrt(shifted_x_p0**2 + shifted_y_p0**2) # lon
shifted_m_lon = np.sqrt(shifted_x_m0**2 + shifted_y_m0**2) # lon 
shifted_p_lat = np.sqrt(shifted_x_p1**2 + shifted_y_p1**2) # lat
shifted_m_lat = np.sqrt(shifted_x_m1**2 + shifted_y_m1**2) # lat

In [134]:
interface = isf_draft*all_cavities_nan
interface_p_lon = ( + interface - np.roll(interface, shift = 1, axis = 0))/dsREF.e2t
interface_m_lon = (- interface + np.roll(interface, shift = -1, axis = 0))/dsREF.e2t
interface_p_lat = (+ interface - np.roll(interface, shift = 1, axis = 1))/dsREF.e1t
interface_m_lat = (- interface + np.roll(interface, shift = -1, axis = 1))/dsREF.e1t
bathymetry = bathymetry*all_cavities_nan
bathymetry_p_lon = ( + bathymetry - np.roll(bathymetry, shift = 1, axis = 0))/dsREF.e2t
bathymetry_m_lon = (- bathymetry + np.roll(bathymetry, shift = -1, axis = 0))/dsREF.e2t
bathymetry_p_lat = (+ bathymetry - np.roll(bathymetry, shift = 1, axis = 1))/dsREF.e1t
bathymetry_m_lat = (- bathymetry + np.roll(bathymetry, shift = -1, axis = 1))/dsREF.e1t
# Take the mean to get the actual local slope (and account for bendy points)
slope_isdraft_lon = np.nansum((interface_p_lon,interface_m_lon), axis = 0)/2
slope_isdraft_lat = np.nansum((interface_p_lat,interface_m_lat), axis = 0)/2
slope_bathy_lon = np.nansum((bathymetry_p_lon,bathymetry_m_lon), axis = 0)/2
slope_bathy_lat = np.nansum((bathymetry_p_lat,bathymetry_m_lat), axis = 0)/2
print('Local meridional and zonal slopes')

Local meridional and zonal slopes


In [135]:
print('If you have calculated the slopes properly, then there shouldn\'t be any nan values')
print('The following numbers should all be 0')
print(np.sum(np.isnan(slope_isdraft_lon)), np.sum(np.isnan(slope_isdraft_lat)), np.sum(np.isnan(slope_bathy_lon)), \
      np.sum(np.isnan(slope_bathy_lat)))

If you have calculated the slopes properly, then there shouldn't be any nan values
The following numbers should all be 0
0 0 0 0


In [136]:
# To calculate the rotated slopes then we need to look at things on a basin by basin basisd
direction_to_front = np.ones(dsREF.nav_lat.data.shape) * np.nan
no_cav_basins = []
#basin_no = 1
for jj in range(len(np.unique(basins_NEMO))):
    basin_no = jj+1
    mask_basin = xr.where(basins_NEMO == basin_no, 1, np.nan)
    x_basin = CC_x[basins_NEMO.values[closed_cavities_nan == 1] == basin_no]
    y_basin = CC_y[basins_NEMO.values[closed_cavities_nan == 1] == basin_no]
    if len(x_basin)>0:
        mean_x_basin = np.mean(x_basin)
        mean_y_basin = np.mean(y_basin)
        nearest_points_i = index[basins_NEMO.values[closed_cavities_nan == 1] == basin_no]
        nearest_x_i = np.ones(len(np.unique(nearest_points_i)))
        nearest_y_i = np.ones(len(np.unique(nearest_points_i)))
        for i in range(len(np.unique(nearest_points_i))):
            nearest_x_i[i], nearest_y_i[i] = OC_x[np.unique(nearest_points_i)[i]], \
                                                   OC_y[np.unique(nearest_points_i)[i]]
        mean_x_i = np.mean(nearest_x_i)
        mean_y_i = np.mean(nearest_y_i)
        direction = np.arctan2(mean_x_basin - mean_x_i, mean_y_basin - mean_y_i)
    else:
        direction = np.nan
        no_cav_basins.append(basin_no)
    #print(jj,len(x_basin), len(nearest_points_i),direction)
    direction_to_front[mask_basin == 1] = np.ones(int(np.nansum(mask_basin.values))) * direction
print('No closed cavities in the following basins:')
print(no_cav_basins)

No closed cavities in the following basins:
[5, 10, 30, 33, 39, 43, 48, 53, 54, 96, 106, 114, 123, 131]


In [137]:
# Then rotate the calculated slopes to have the slope towards the ice front and the slope across the ice front 
slope_isdraft_hypot = np.sqrt(slope_isdraft_lon **2 + slope_isdraft_lat **2)
angle_isdraft = np.arctan2(slope_isdraft_lon,slope_isdraft_lat)- direction_to_front
slope_is_across_front = slope_isdraft_hypot * np.sin(angle_isdraft)
slope_is_towards_front = slope_isdraft_hypot * np.cos(angle_isdraft)
slope_bathy_hypot = np.sqrt(slope_bathy_lon **2 + slope_bathy_lat **2)
angle_bathy = np.arctan2(slope_bathy_lon,slope_bathy_lat)- direction_to_front
slope_ba_across_front = slope_bathy_hypot * np.sin(angle_bathy)
slope_ba_towards_front = slope_bathy_hypot * np.cos(angle_bathy)
print('Rotated slopes')

Rotated slopes


In [138]:
print('If you have calculated the slopes properly, then there shouldn\'t be any nan values')
print('The following numbers should all be 0')
print(np.sum(np.isnan(slope_is_across_front[closed_cavities == 1])), np.sum(np.isnan(slope_is_towards_front[closed_cavities == 1])), 
      np.sum(np.isnan(slope_ba_across_front[closed_cavities == 1])), np.sum(np.isnan(slope_ba_towards_front[closed_cavities == 1])))

If you have calculated the slopes properly, then there shouldn't be any nan values
The following numbers should all be 0
0 0 0 0


In [139]:
filepath_mask = filepath_base +  'AIAI_data/Masks/'+nemo_run+'_'+'geometric_masks.nc'
# A geometry file with the required variables for the NN 
filepath_geomvars = filepath_base + 'AIAI_data/Masks/'+nemo_run+'_'+'geom_vars.nc'

In [140]:
filepath_mask

'/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Masks/isfru94_geometric_masks.nc'

In [141]:
# Save one file for the 
ds = xr.Dataset(
        data_vars=dict(
            ocean_only              = (["y", "x"], ocean.data),
            ocean_only_nan          = (["y", "x"], ocean_nan.data),
            all_cavities            = (["y", "x"], all_cavities.data),
            all_cavities_nan        = (["y", "x"], all_cavities_nan.data),
            closed_cavities         = (["y", "x"], closed_cavities.data),
            closed_cavities_nan     = (["y", "x"], closed_cavities_nan.data),
            open_cavities           = (["y", "x"], open_cavities.data),
            open_cavities_nan       = (["y", "x"], open_cavities_nan.data),
            ocean_open_cavities     = (["y", "x"], open_cav_ocean.data),
            ocean_open_cavities_nan = (["y", "x"], open_cav_ocean.data),
            grounded_ice            = (["y", "x"], grounded_ice.data),
            grounded_ice_nan        = (["y", "x"], grounded_ice_nan.data),    
            basins_NEMO             = (["y", "x"], basins_NEMO.data),
            mask_TS_profiles        = (["y", "x"], mask_TS_profiles.data),
            mask_TS_profiles_nan    = (["y", "x"], mask_TS_profiles_nan.data),
            ),
        coords=dict(
            lon=(("y", "x"), dsREF.nav_lon.data),
            lat=(("y", "x"), dsREF.nav_lat.data),    ),
        attrs=dict(description="Masks for grounded ice, floating ice and open ocean", 
                   simulation_run = 'Christoph_CFG'),
    )
ds.to_netcdf(filepath_mask)
print('You have saved the following file')
print(filepath_mask)

You have saved the following file
/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Masks/isfru94_geometric_masks.nc


In [142]:
# Save one file for the 
ds = xr.Dataset(
        data_vars=dict(
            ocean_only              = (["y_grid_T", "x_grid_T"], ocean.data),
            ocean_only_nan          = (["y_grid_T", "x_grid_T"], ocean_nan.data),
            all_cavities            = (["y_grid_T", "x_grid_T"], all_cavities.data),
            all_cavities_nan        = (["y_grid_T", "x_grid_T"], all_cavities_nan.data),
            closed_cavities         = (["y_grid_T", "x_grid_T"], closed_cavities.data),
            closed_cavities_nan     = (["y_grid_T", "x_grid_T"], closed_cavities_nan.data),
            open_cavities           = (["y_grid_T", "x_grid_T"], open_cavities.data),
            open_cavities_nan       = (["y_grid_T", "x_grid_T"], open_cavities_nan.data),
            ocean_open_cavities     = (["y_grid_T", "x_grid_T"], open_cav_ocean.data),
            ocean_open_cavities_nan = (["y_grid_T", "x_grid_T"], open_cav_ocean.data),
            grounded_ice            = (["y_grid_T", "x_grid_T"], grounded_ice.data),
            grounded_ice_nan        = (["y_grid_T", "x_grid_T"], grounded_ice_nan.data),    
            basins_NEMO             = (["y_grid_T", "x_grid_T"], basins_NEMO.data),
            mask_TS_profiles        = (["y_grid_T", "x_grid_T"], mask_TS_profiles.data),
            mask_TS_profiles_nan    = (["y_grid_T", "x_grid_T"], mask_TS_profiles_nan.data),
            ),
        coords=dict(
            lon=(("y_grid_T", "x_grid_T"), dsREF.nav_lon.data),
            lat=(("y_grid_T", "x_grid_T"), dsREF.nav_lat.data),    ),
        attrs=dict(description="Masks for grounded ice, floating ice and open ocean", 
                   simulation_run = 'Christoph_CFG'),
    )
ds.to_netcdf(filepath_mask)
print('You have saved the following file')
print(filepath_mask)

You have saved the following file
/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Masks/isfru94_geometric_masks.nc


In [143]:
# And then save these to file 
ds = xr.Dataset(
            data_vars=dict(
                areas                  = (["y", "x"], areas.data),
                bathymetry             = (["y", "x"], bathymetry.data),
                isf_draft              = (["y", "x"], isf_draft.data),
                distances_GL           = (["y", "x"], distances_GL.data),
                distances_OO           = (["y", "x"], distances_OO.data),
                distances_OC           = (["y", "x"], distances_OC.data),
                slope_isdraft_lon      = (["y", "x"], slope_isdraft_lon.data),
                slope_isdraft_lat      = (["y", "x"], slope_isdraft_lat.data),
                slope_bathy_lon        = (["y", "x"], slope_bathy_lon.data),
                slope_bathy_lat        = (["y", "x"], slope_bathy_lat.data),
                slope_is_across_front  = (["y", "x"], slope_is_across_front.data),
                slope_is_towards_front = (["y", "x"], slope_is_towards_front.data),
                slope_ba_across_front  = (["y", "x"], slope_ba_across_front.data),
                slope_ba_towards_front = (["y", "x"], slope_ba_towards_front.data),
                ),
            coords=dict(
                lon=(("y", "x"), dsREF.nav_lon.data),
                lat=(("y", "x"), dsREF.nav_lat.data),    ),
            attrs=dict(description = "Required geometry variables for this geometry", 
                       simulation_run = 'Christoph_CFG'),
                )
ds.to_netcdf(filepath_geomvars)
print('You have saved the following file')
print(filepath_geomvars)

You have saved the following file
/bettik/ockendeh/SCRIPTS/simpleNN_basal_melt/AIAI_data/Masks/isfru94_geom_vars.nc
